In [13]:
# -*- coding: utf8 -*-

"""Contains function to perform GX-FBA"""

from __future__ import absolute_import

import six
from optlang.symbolics import Zero
from cobra.flux_analysis import flux_variability_analysis as fva


def gx_fba(model, expression_profile, conditon=None, reg_threshold=0.5, flux_threshold=100.0):
    """
    GX-FBA[1]
    
    Parameters
    ----------
    model: cobra.Model
        The constraint-based model to perform GX-FBA on.
    expression_profile: ExpressionProfile
        The expression profile to integrate in the model.
    conditions: str or int, optional
        The condition from the expression profile.
    reg_threshold: float, optional (default 0.5)
        The minumum fold change to be considered for up or down regulation.
    flux_threshold: float, optional (default 100.0)
        The maximum flux value to be considered for a reaction. if not obeyed
        by reaction, it is discarded from analysis.
    
    Returns
    -------
    model, cobra.Solution
    
    References
    ----------
    .. [1] Navid and Almaas (2012).
           Genome-level transcription data of Yersinia pestis analyzed with a New metabolic constraint-based approach.
           BMC Systems Biology 2012 6:150.
           doi:10.1186/1752-0509-6-150
    """
    
    assert isinstance(model, cobra.Model)
    assert isinstance(expression_profile, ExpressionProfile)
    assert isinstance(condition, (str, int, None))
    assert isinstance(threshold, float)
    
    with model:
        prob = model.problem
        # Wild-type flux distribution
        sol_wt = model.optimize()
        # Flux variability analysis
        fva_sol = fva(model)
        # Mean possible flux for each reaction
        fva_mean = fva_sol.mean(axis=1)
        # Avg. flux carried by active reactions
        avg_flux = fva_mean.mean()
        
        condition = expression_profile.conditions[0] if condition is None else condition
        reaction_profile = expression_profile.to_reaction_dict(condition, model, not_measured_value, normalization)
        
        obj_vars = []
        for rxn_id, rxn_exp in six.iteritems(reaction_profile):
            rxn = model.reactions.get_by_id(rxn_id)
            if rxn not in model.exchanges and rxn.reversibility is False \
                and fva_mean[rxn_id] > flux_threshold:
                if rxn_exp > 0:
                    rxn.upper_bound = rxn_exp * fva_mean[rxn_id]
                elif rxn_exp < 0:
                    rxn.lower_bound = rxn_exp * fva_mean[rxn_id]
                
                obj_var = prob.Variable(np.log2(rxn_exp) * (rxn.flux_expression / fva_mean[rxn_id]))
                obj_vars.append(var)
                
        model.add_cons_vars(obj_vars)
        model.objective = prob.Objective(Zero, direction="max", sloppy=True)
        model.objective.set_linear_coefficients({v: 1.0 for v in obj_vars})
        
        solution = model.optimize()
        
        return (model, solution)

In [1]:
import cobra.test

In [2]:
import cobra

In [3]:
model = cobra.test.create_test_model("textbook")

In [4]:
sol_wt = model.optimize()

In [5]:
fva_sol = cobra.flux_analysis.flux_variability_analysis(model)

In [6]:
sol_wt

,fluxes,reduced_costs
ACALD,0.000000,6.938894e-18
ACALDt,0.000000,0.000000e+00
ACKr,0.000000,1.040834e-17
ACONTa,6.007250,0.000000e+00
ACONTb,6.007250,1.387779e-17
...,...,...
TALA,1.496984,-2.775558e-17
THD2,0.000000,-2.546243e-03
TKT1,1.496984,-1.387779e-17
TKT2,1.181498,1.387779e-17


In [7]:
fva_mean = fva_sol.mean(axis=1)

In [8]:
avg_flux = fva_mean.mean()

In [9]:
import numpy as np

In [11]:
np.log2([0.1, 2.5, 7.3])

array([-3.32192809,  1.32192809,  2.86789646])

In [12]:
model.reactions.ACALDt not in model.exchanges

True